In [1]:
%reset
import importlib
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
import time
import networkx as nx
import osmnx as ox
from itertools import permutations
# For base case simulation
import baseCaseSimulation as baseCase
from baseCaseSimulation import *
import vehicleMatching as vm
from vehicleMatching import *
# For simulation case
import UPDATE_RouteSimClass as simul
from UPDATE_RouteSimClass import *
import UPDATE_RouteRiderVanClass as rv
from UPDATE_RouteRiderVanClass import *
os.chdir("C:/Users/Rick/Desktop/Research/RideACTA_RAMP/Data/RideACTA/")

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
%%time
# ---------------------------- Load Data ----------------------------------------------

#    Load in all the trajectory and ridership data for May 1, 2019 - Aug 1, 2019

# -------------------------------------------------------------------------------------
trajFile = "1b_filteredTrajWithFreeway.csv"
riderFile = '2f_finalRidershipHighwayFilter.csv'
travelTimesFile = '3e_medTravTimes.csv'

# Load trajectories for base case simulation
def loadTrajForBaseCase(trajFile):
    trajData = pd.read_csv(trajFile)
    trajData = trajData.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)
    trajData = trajData[trajData['month'].isin([5, 6, 7])]
    trajData['datetime'] = pd.to_datetime(trajData['datetime'])
    return trajData

def loadRiderData(riderFile):
    riderData = pd.read_csv(riderFile)
    riderData = riderData.drop('Unnamed: 0', axis = 1)
    riderData = riderData[riderData['month'].isin([5, 6, 7])]
    riderData['destination_poi'] = riderData['destination_poi'].astype(int)
    riderData['origin_timestamp'] = pd.to_datetime(riderData['origin_timestamp'])
    riderData['pending_time'] = pd.to_datetime(riderData['pending_time'])
    riderData['pending_sec'] = riderData['pending_time'].apply(lambda row: row.hour*3600 + row.minute*60 + row.second)
    riderData['destination_poi'] = riderData['destination_poi'].astype(int)
    riderData['od_pair'] = riderData['origin_poi'].astype(str) + ', ' + riderData['destination_poi'].astype(str)
    riderData['vehicle'] = riderData['vehicle'].astype(int)
    deltaTime2 = 15
    riderData['timePeriod1'] = riderData['origin_timestamp'].apply(lambda row: dt.datetime(row.year, row.month, row.day, row.hour, \
                                                             deltaTime2*(row.minute // deltaTime2))).dt.strftime('%H:%M:%S')
    return riderData

def loadTravTimesForRuleBasedSimulation(travelTimesFile):
    travTimes = pd.read_csv(travelTimesFile)
    travTimes = travTimes.drop(['Unnamed: 0'], axis = 1)
    travTimes['median_x'] = travTimes['median_x'].astype(int)
    new = travTimes['OD_string'].str.split(', ', n=1, expand=True)
    travTimes['POI'] = new[0].astype(int)
    travTimes['dest_POI'] = new[1].astype(int)
    return travTimes

def createGraph(travTimes, currentTimePeriod):
    baseTime = '6:00:00'
    # create column of time differences between current time and travel time
    travTimes.loc[:,'currentTime'] = abs((pd.to_timedelta(currentTimePeriod).total_seconds() - \
                                                pd.to_timedelta(baseTime).total_seconds()) - \
                                               (pd.to_timedelta(travTimes.loc[:,'timePeriod']).dt.total_seconds() - \
                                                pd.to_timedelta(baseTime).total_seconds()))
    # Find the closest time (either before or after) to determine travel time between two given nodes
    a = travTimes.groupby(['OD_string']).apply(lambda travTimes: travTimes['currentTime'].idxmin())
    # Filter the travel times based on the time-dependent travel times
    timeDependentTravTimes = travTimes.loc[a, :].reset_index(drop=True)
    # Create the graph for the specific time-dependent travel times
    G = nx.from_pandas_edgelist(timeDependentTravTimes, source = 'POI', target = 'dest_POI', \
                                edge_attr = 'median_x', create_using = nx.DiGraph())
    return G

# Create time dependent network from median travel times
def createTravTimeDict(travTimes):
    timeDepTravTime = dict()
    timeList1 = [f'0{hour}:{minute}:00' for hour in list(range(6,10)) for minute in ('00','15','30','45')]
    timeList2 = [f'{hour}:{minute}:00' for hour in list(range(10,24)) for minute in ('00','15','30','45')]
    timeList3 = timeList1 + timeList2
    for currentTime in timeList3:
        G = createGraph(travTimes, currentTime)
        timeDepTravTime[currentTime] = G
    return timeDepTravTime

trajData = loadTrajForBaseCase(trajFile)
riderData = loadRiderData(riderFile)
travTimes = loadTravTimesForRuleBasedSimulation(travelTimesFile)
timeDepTravTime = createTravTimeDict(travTimes)
newDF = riderData[['ride_id', 'orig_sec']].copy()
origTimeDict = newDF.set_index('ride_id').to_dict()
riderTimeDict = origTimeDict['orig_sec']

Wall time: 1min 1s


The following cell is used to run the base case simulation based on actual vehicle trajectories. This is very approximate and some of the requests will have to be dropped from the analysis based on infeasible pickup/dropoff locations

In [3]:
# Do base case simulation
import baseCaseSimulation as baseCase
from baseCaseSimulation import *
import vehicleMatching as vm
from vehicleMatching import *

importlib.reload(baseCase)
importlib.reload(vm)

# Run simulation for weekdays in July 2019
simulDates = ['2019-07-01','2019-07-03','2019-07-05',\
             '2019-07-08','2019-07-09','2019-07-10','2019-07-11','2019-07-12',\
             '2019-07-15','2019-07-16','2019-07-17','2019-07-18','2019-07-19',\
             '2019-07-22','2019-07-23','2019-07-24','2019-07-25','2019-07-26']

# Van Ids from Traj File
trajIkeaVehList = [1237, 1533, 1535]
# Van Ids from Rider File
riderIkeaVehList = [2008, 2064, 2068]
# Run Simulation
baseCaseResults, requestsDict = runBaseCaseForMultipleDates(trajData, riderData, simulDates, riderIkeaVehList, trajIkeaVehList)


The following code block defines the simulation scenario. This is based on vans making decisions based on the minimal marginal cost for each additional rider.

In [4]:
import UPDATE_RouteSimClass as simul
from UPDATE_RouteSimClass import *
import UPDATE_RouteRiderVanClass as rv
from UPDATE_RouteRiderVanClass import *

importlib.reload(simul)
importlib.reload(rv)

# Calculate haversine distances
def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))*3.28084 #convert to feet

# Find van start times for the ikea vehicles
def findIkeaStartTime(trajInfo):
    ikea = (40.452828, -80.168373)
    vanStartTime = dict()
    for vehicle in trajInfo['vehicle_id'].unique():
        filtData = trajInfo[trajInfo['vehicle_id'] == vehicle].sort_values(by='sec').reset_index(drop=True).copy()
        for index, row in filtData.iterrows():
            if (haversine(ikea, (row['latitude'], row['longitude'])) < 1500):
                vanStartTime[vehicle] = row['sec']
                break
    return vanStartTime

def selectSimDate(simulationDate, riderData, requestsToConsider):
    westBusNodes = [249, 197, 221, 200, 238, 124, 107, 116, 122, 124, 63, 113, 104, 227, 163, 84, 157, 110, 186, 14, 204]
    # Filter data to only include IKEA riders
    dfIKEA = riderData[(riderData['date'] == simulationDate) & \
                       (riderData['ride_id'].isin(requestsToConsider)) & \
                       (riderData['vehicle'].isin([2008, 2064, 2068])) & \
                       (~riderData['origin_poi'].isin(westBusNodes)) & \
                       (~riderData['destination_poi'].isin(westBusNodes))].reset_index(drop=True)
    
    # A couple of nodes are not present in the graph, so we can replace missing nodes with nearby nodes
    dfIKEA[['origin_poi','destination_poi']] = dfIKEA[['origin_poi','destination_poi']].replace(46, 39)
    dfIKEA[['origin_poi','destination_poi']] = dfIKEA[['origin_poi','destination_poi']].replace(33, 3)
    #dfIKEA[['origin_poi','destination_poi']] = dfIKEA[['origin_poi','destination_poi']].replace(7, 214)
    # This was visited once during a 4 month period, so we can drop this
    dfIKEA1 = dfIKEA[(dfIKEA['origin_poi'] != 67) & (dfIKEA['destination_poi'] != 67) & \
                     (dfIKEA['origin_poi'] != 7) & (dfIKEA['destination_poi'] != 7) & \
                     (dfIKEA['origin_poi'] != 275) & (dfIKEA['destination_poi'] != 275)].copy()
    
    print(f'Count of trips dropped because node was not present in graph: {len(dfIKEA) - len(dfIKEA1)}') 
    dfIKEA2 = dfIKEA1[['ride_id','date','vehicle','origin_timestamp','timePeriod1','orig_sec','pending_sec', \
                       'origin_poi','destination_poi','od_pair', 'USE_orig_lat',
                       'USE_orig_long', 'USE_dest_lat', 'USE_dest_long']].sort_values(by='origin_timestamp').copy()
    return dfIKEA2

# Create a list of rider objects to serve during the day
def createDailyRiderList(riderData):
    riderList = []
    for i in range(len(riderData)):
        riderList.append(rider(riderData['ride_id'].iloc[i], riderData['orig_sec'].iloc[i], 
                               riderData['pending_sec'].iloc[i], riderData['origin_poi'].iloc[i], 
                               riderData['destination_poi'].iloc[i], riderData['timePeriod1'].iloc[i],
                               riderData['USE_orig_lat'].iloc[i], riderData['USE_orig_long'].iloc[i],
                               riderData['USE_dest_lat'].iloc[i], riderData['USE_dest_long'].iloc[i]))
    # create a list of riders that need to be served where riders can be removed throughout the day
    remainingRiders = riderList.copy()
    return riderList, remainingRiders

# Get dictionary of IKEA vehicles { vehID : startTime }
def getIkeaDict(vehDict):
    newDict = dict()
    for key, value in vehDict.items():
        if (key in [1237, 1533, 1535]):
            newDict[key] = value
    return newDict

def storeSimulationResults(dailyRiders, riderData, date):
    # Filter original rider data
    dfVan = riderData[(riderData['date'] == date)].copy()
    dfVan['origin_timestamp'] = dfVan['origin_timestamp'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
    # Convert new dataframe from the simulation results
    a,b,c,d =[],[],[],[]
    for rider in dailyRiders:
        a.append(rider.van)
        b.append(rider.pickupTime)
        c.append(rider.dropoffTime)
        d.append(rider.rideID)
    x = pd.DataFrame(data = {'ride_id':d,'vanID':a, 'pickup_Sim':b,'dropoff_Sim':c})
    mergeDF = pd.merge(dfVan, x, how = 'left', on = 'ride_id')
    mergeDF1 = mergeDF[['ride_id','vehicle','vanID','origin_timestamp','orig_sec', 'pickup_Sim','dropoff_Sim','od_pair']]
    mergeDF1.loc[:,'waitTimeSim'] = mergeDF1['pickup_Sim'] - mergeDF1['orig_sec']
    mergeDF1.loc[:,'driveTimeSim'] = mergeDF1['dropoff_Sim'] - mergeDF1['pickup_Sim']
    return mergeDF1

def filtTrajData(trajData, simulDate):
    traj = trajData[trajData['date'] == simulDate].reset_index(drop=True).copy()
    vanStartTime = findIkeaStartTime(traj)
    ikeaVehs = getIkeaDict(vanStartTime)
    return ikeaVehs

def runDaySimulation(riderData, date, timeDepTravTime, riderTimeDict, requestsDict, vehCount, timeWindow):
    resultsDF = pd.DataFrame(columns = ['ride_id','vehicle','vanID','origin_timestamp','orig_sec','pickup_Sim',\
                                        'dropoff_Sim','od_pair','waitTimeSim','driveTimeSim'])
    df = selectSimDate(date, riderData, requestsDict[date])
    # =================== Run simulation Section ====================
    # Compile list of riders for a specific day assigned to a specific van
    dailyRiders, remainingRiders = createDailyRiderList(df)
    
    # For varying number of vans
    if (vehCount == 0):
        ikeaVehs = filtTrajData(trajData, date)
        simStartTime = min([value for key, value in ikeaVehs.items()])
    elif (vehCount == 1):
        ikeaVehs = dict([(1535, 6*3600)])
        simStartTime = 6*3600
    elif (vehCount == 2):     
        ikeaVehs = dict([(1535, 6*3600), (1533, 7*3600)])
        simStartTime = 6*3600
       
    # ------------------------------------
    # Inputs for simulation
    waitCoeff = 0.75
    driveCoeff = 0.25
    deltaTime = 0*60
    
    sim = Sim(simStartTime, ikeaVehs, remainingRiders, timeDepTravTime, riderTimeDict, waitCoeff, driveCoeff, timeWindow) 
    while (sim.time <= 24*3600-60):
        # Assign working vans based on their start times
        sim.activateVans()
        sim.assignRiderToVan()
        sim.nextTrip()
        sim.moveVans()
        sim.step()

    storedResults = storeSimulationResults(dailyRiders, df, date)
    return storedResults

# Run simulation for multiple dates
def runSimulationForMultipleDates(trajData, riderData, dates, riderTimeDict, requestsDict, vehCount, timeWindow):
    count = 0
    for simulDate in dates:
        print(simulDate)
        if (count == 0):
            simulationResults = runDaySimulation(riderData, simulDate, timeDepTravTime, riderTimeDict, requestsDict, vehCount, timeWindow)
            count += 1
        else:
            simulationResults1 = runDaySimulation(riderData, simulDate, timeDepTravTime, riderTimeDict, requestsDict, vehCount, timeWindow)
            simulationResults = simulationResults.append([simulationResults1],sort=True)
    return simulationResults


In [4]:
# Run simulation for dates in July 2019
simulDates = ['2019-07-01','2019-07-03','2019-07-05',\
             '2019-07-08','2019-07-09','2019-07-10','2019-07-11','2019-07-12',\
             '2019-07-15','2019-07-16','2019-07-17','2019-07-18','2019-07-19',\
             '2019-07-22','2019-07-23','2019-07-24','2019-07-25','2019-07-26']

vehCount = 2
timeWindow = 0*60
simulationResults = runSimulationForMultipleDates(trajData, riderData, simulDates, riderTimeDict, requestsDict, vehCount, timeWindow)

#For multiple vans
# i = 0
# for vehCount in [1,2,3]:
#     print(vehCount)
#     if (i == 0):
#         simulationResults = runSimulationForMultipleDates(trajData, riderData, simulDates, riderTimeDict, requestsDict, vehCount, timeWindow)
#         simulationResults.loc[:, 'vanCount'] = vehCount
#     else:
#         simulationResults1 = runSimulationForMultipleDates(trajData, riderData, simulDates, riderTimeDict, requestsDict, vehCount,timeWindow)
#         simulationResults1.loc[:, 'vanCount'] = vehCount
#         simulationResults = simulationResults.append([simulationResults1],sort=True)
#     i += 1

#simulationResults.to_csv('1van.csv')
#simulationResults.to_csv('USE_1van_uber_6min.csv')
#simulationResults.to_csv('USE_2van.csv')

The next code block is used to post-process data to include TNC assignments. During the simulation, a rider is assigned a TNC if their estimated wait time > wait time threshold defined in model. Wait and drive time are then added to results for TNC trips based on travel time graph and average wait times for Pittsburgh for Uber.

In [5]:
# Load in simulation results
# For 1 van case
van_1_uber = pd.read_csv('USE_1van_uber_6min.csv')
van_1_uber.loc[:,'type'] = 1
van_1_uber.loc[:,'ones'] = 0
# For 2 van case
van_2 = pd.read_csv('USE_2van.csv')
van_2.loc[:,'type'] = 2
# For uber only case
uber_only = van_2[['ride_id','od_pair','orig_sec','origin_timestamp']].copy()
uber_only.loc[:,'type'] = 3
uber_only.loc[:,'waitTimeSim'] = 5*60
uber_only.loc[:,'driveTimeSim'] = 0
uber_only.loc[:,'vanID'] = 'Uber'
new = uber_only['od_pair'].str.split(',', n = 1, expand = True) 
uber_only['orig'] = new[0]
uber_only['dest'] = new[1]
uber_only['dest'] = uber_only['dest'].str.strip()
uber_only[['orig','dest']] = uber_only[['orig','dest']].replace('46', '39')
uber_only[['orig','dest']] = uber_only[['orig','dest']].replace('33', '3')

def getTimePeriod(timeString):
        timeInterval = 15
        timePeriod = dt.datetime.strptime(timeString, '%H:%M:%S').time()
        timePeriod = str(timePeriod.replace(minute = timeInterval*(timePeriod.minute//timeInterval), second = 0))
        return timePeriod
    
def convertSecondsToTimeString(seconds):
    hours = str(int(seconds//3600))
    minutes = str(int((seconds%3600)//60))
    if len(hours) < 2:
        hours = '0' + hours
    if len(minutes) < 2:
        minutes = '0' + minutes
    secs = '00'
    return ':'.join([hours, minutes, secs])

# For Uber + Van case
for i in range(len(van_1_uber)):
    if (van_1_uber['vanID'].iloc[i] == 'Uber'):
        van_1_uber['pickup_Sim'].iloc[i] = van_1_uber['orig_sec'].iloc[i] + 5*60
        van_1_uber['waitTimeSim'].iloc[i] = 5*60
        van_1_uber['ones'].iloc[i] = 1
        timeString = convertSecondsToTimeString(uber_only['orig_sec'].iloc[i])
        timePeriod = getTimePeriod(timeString)
        van_1_uber['driveTimeSim'].iloc[i] = nx.shortest_path_length(timeDepTravTime[timePeriod], \
                                                                 source = int(uber_only['orig'].iloc[i]), \
                                                                 target = int(uber_only['dest'].iloc[i]), \
                                                                 weight = 'median_x') 

# For Uber Only Case
for i in range(len(uber_only)):
    timeString = convertSecondsToTimeString(uber_only['orig_sec'].iloc[i])
    timePeriod = getTimePeriod(timeString)
    uber_only['driveTimeSim'].iloc[i] = nx.shortest_path_length(timeDepTravTime[timePeriod], \
                                                             source = int(uber_only['orig'].iloc[i]), \
                                                             target = int(uber_only['dest'].iloc[i]), \
                                                             weight = 'median_x')
mergeDF1 = van_2.append([van_1_uber, uber_only], sort=True)

The next code block is to compute agency costs for each TNC trip based on Uber costs estimates for the region. The cost is determined by a function that includes driving distance, driving time, and base fare.

In [6]:
travDist = pd.read_csv('Robinson_Trav_Dist.csv')

def distDict(data):
    storeDict = dict()
    for i in range(len(data)):
        orig = data['Orig_Node'].iloc[i]
        dest = data['Dest_Node'].iloc[i]
        storeDict[f'{orig}, {dest}'] = data['dist'].iloc[i]
        storeDict[f'{dest}, {orig}'] = data['dist'].iloc[i]
    return storeDict

def calcCost(timeMin, distMile):
    minFare = 8
    baseFare = 1.43
    bookingFee = 2.5
    cost = baseFare + bookingFee + (0.32*timeMin) + (0.87*distMile)
    totalFee = max(minFare, cost)
    return totalFee
    
def addUberCosts(data, dist):
    data.loc[:,'uberCost'] = 0
    for i in range(len(data)):
        if (data['vanID'].iloc[i] == 'Uber'):
            timeMin = data['driveTimeSim'].iloc[i]/60
            distMile = dist.get(data['od_pair'].iloc[i], 10)
            data['uberCost'].iloc[i] = calcCost(timeMin, distMile)
    return data
    
dist = distDict(travDist)
df_costs = addUberCosts(mergeDF1, dist)
df_costs['origin_timestamp'] = df_costs['origin_timestamp'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
df_costs.loc[:,'date'] = df_costs['origin_timestamp'].dt.date.astype(str)
df_costs.loc[:,'hour'] = df_costs['origin_timestamp'].dt.hour
grouped = df_costs.groupby(['date','type'])['uberCost'].sum().reset_index()

# Assign van costs ($51/hour worked)
grouped.loc[:, 'totalCost'] = 0
grouped.loc[grouped['type'] == 1, 'totalCost'] = grouped['uberCost'] + (18*51)
grouped.loc[grouped['type'] == 2, 'totalCost'] = (14*51) + (17*51)
grouped.loc[grouped['type'] == 3, 'totalCost'] = grouped['uberCost']
grouped.loc[grouped['type'] == 1, 'Case'] = '1 Van + TNC'
grouped.loc[grouped['type'] == 2, 'Case'] = '2 Vans'
grouped.loc[grouped['type'] == 3, 'Case'] = 'TNC Only'

In [16]:
# Process simulation data to format for plotting
def processSimulationData(simulationResults1):
    simulCase = simulationResults1[['ride_id','od_pair','origin_timestamp','vanID','pickup_Sim','dropoff_Sim','waitTimeSim', \
                                    'driveTimeSim','type']].copy()
    simulCase['origin_timestamp'] = simulCase['origin_timestamp'].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
    simulCase.loc[:, 'hour'] = simulCase['origin_timestamp'].dt.hour
    simulCase.loc[(simulCase['waitTimeSim'] < 0), 'waitTimeSim'] = 0
    simulCase['waitTimeSim'] = simulCase['waitTimeSim'].astype(float)
    simulCase['driveTimeSim'] = simulCase['driveTimeSim'].astype(float)
    simulCase = simulCase.rename(columns={'waitTimeSim':'simulWaitTime', 'driveTimeSim':'simulDriveTime'})
    simulCase.loc[:,'date'] = simulCase['origin_timestamp'].dt.date.astype(str)
    simulCase.loc[:,'userCost'] = simulCase['simulWaitTime']*0.75 + simulCase['simulDriveTime']*0.25
    return simulCase

#plotBaseCaseData = processBaseCaseData(baseCaseResults)
plotSimulData = processSimulationData(mergeDF1)
#mergeResults = pd.merge(plotSimulData, plotBaseCaseData, how = 'left', on = 'ride_id')
plotting = plotSimulData[['od_pair','hour','simulWaitTime','simulDriveTime','type','userCost']].groupby(['hour','type']).sum()/len(simulDates)
plotting = plotting.reset_index()